In [2]:
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from time import sleep
import openpyxl
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
import time
import os
from tabula import read_pdf
import pdfplumber
from time import sleep
import pypdfium2 as pdfium
import openpyxl

In [3]:
def login_commec(browser,username,password):
    browser.get("https://www2.commsec.com.au/secure/login/netbank")
    time.sleep(2)
    kw_input = browser.find_element(By.XPATH, '//*[@id="username"]')
    kw_input.click()
    time.sleep(1)
    kw_input.send_keys(username)
    time.sleep(1)
    kw_input = browser.find_element(By.XPATH, '//*[@id="password"]')
    kw_input.click()
    kw_input.send_keys(password)
    browser.find_element(By.XPATH, '//*[@id="login"]').click()
    time.sleep(2)

In [4]:
def get_last_file(download_dir):
    file_list = os.listdir(download_dir)
    file_list.sort(key=lambda file: os.path.getmtime(os.path.join(download_dir,file)))
    return file_list[-1]

In [5]:
def renamepdf(new_file_name, download_dir):
    get_last_file(download_dir)
#     new_file_name = 'Trading-Halt ('+ index +').PDF'
    os.rename(os.path.join(download_dir,get_last_file(download_dir)), os.path.join(download_dir,new_file_name))

In [6]:
def find_word(path, key_works):
    pdf = pdfium.PdfDocument(path)
#     key_works = ['placement','raising','SPP','share purchase plan', 'IPO', 'Right issue']
    get_word = 0
    for page in pdf:
        textpage = page.get_textpage()
        text_all = textpage.get_text_range()
        for key_work in key_works:
            searcher = textpage.search(key_work, match_case=False, match_whole_word=False)
            first_occurrence = searcher.get_next()
            if first_occurrence != None:
#                 print('ok')
                get_word = 1
                break
#             else:
#                 print('no')
        if get_word == 1:
            break
    return get_word

In [7]:
def check_file(file_name ,file_index, path, download_dir, last_pdf):
    file_intact = file_name.format(file_index = str(file_index))
    while file_intact not in path:
                    time.sleep(1)
                    try:
                        path1 = os.listdir(download_dir)
                        now_pdf = get_last_file(download_dir)
                        if now_pdf != last_pdf:
                            time.sleep(1)
                            renamepdf(file_intact, download_dir)
                    except Exception as error:
                        print("An exception occurred:", error)
                    if file_intact in path1:
                        break

In [8]:
def write_excel(info,excel_dir, sheet_name):
    writer = pd.ExcelWriter(excel_dir)
    data = pd.DataFrame(info)
    data.to_excel(writer, sheet_name=sheet_name, index=False, header=False)
    writer.save()
    print("xlsx write successfully！")

In [18]:
def add_to_excel(data_info, excel_dir, sheet_name):
    writer = pd.ExcelWriter(excel_dir, mode='a', engine='openpyxl',if_sheet_exists='overlay')
    df1 = pd.DataFrame(pd.read_excel(excel_dir, sheet_name=sheet_name)) 
    df_cols = df1.shape[1] 
    data = pd.DataFrame(data_info)
    data.to_excel(writer, sheet_name=sheet_name,startcol=df_cols+1, index=False, header=False)
    writer.save()
    print("xlsx write successfully！")

In [10]:
def get_company(browser):
    action = ActionChains(browser)
    soup = BeautifulSoup(browser.page_source, 'lxml')
    products = soup.find('div',class_='dailyTable')
    products1 = products.find('tbody')
    i=0
    j=1
    index = 1
    mat = [['Company Name','Ticker','Last price','Market Cap','Last price_d','Market Cap_d']]
    for product in products1:
        ln = []
        if('flow' in product.find_all('td')[3].text.lower() or '5b' in product.find_all('td')[3].text.lower() or '4c' in product.find_all('td')[3].text.lower()):
            i=i+1
            print(i)
            action.move_to_element(browser.find_element(By.XPATH,'//*[@id="module_8"]/section/div/div/div[2]/div/div[3]/table/tbody/tr['+ str(index) +']/td[1]/a')).perform()
            time.sleep(2)
            soup1 = BeautifulSoup(browser.page_source, 'lxml').find('div',class_='dailyTable').find('tbody').find_all('tr')[index-1]
            print(soup1.find_all('td')[0].find('div').find_all('tr')[0].find_all('td')[1].text)
            print(soup1.find_all('td')[0].find('div').find_all('tr')[6].find_all('td')[1].text.strip('$').replace(' ', ''))
            print(soup1.find_all('td')[0].find('div').find_all('div')[1].find_all('div')[1].find('div').find_all('a')[2].get('href'))

            print(product.find_all('td')[0].text,'  ',product.find_all('td')[1].text, ' ', 'https://research.commsec.com.au'+product.find_all('td')[3].find('a').get('href'), ' ')

            ln.append(product.find_all('td')[1].text)
            ln.append(product.find_all('td')[0].text)
            try:
                ln.append(soup1.find_all('td')[0].find('div').find_all('tr')[0].find_all('td')[1].text)
                ln.append('$'+soup1.find_all('td')[0].find('div').find_all('tr')[6].find_all('td')[1].text.strip('$').replace(' ', ''))
                ln.append(float(soup1.find_all('td')[0].find('div').find_all('tr')[0].find_all('td')[1].text.replace('$', '')))
                aa = soup1.find_all('td')[0].find('div').find_all('tr')[6].find_all('td')[1].text.replace('$', '')
                if('M' in soup1.find_all('td')[0].find('div').find_all('tr')[6].find_all('td')[1].text.replace('$', '')):
                    aa = float(soup1.find_all('td')[0].find('div').find_all('tr')[6].find_all('td')[1].text.replace('$', '').replace('M', ''))
                ln.append(aa)
            except Exception as error:
                print("An exception occurred:", error)
                ln.append(error)
            ln.append(soup1.find_all('td')[0].find('div').find_all('div')[1].find_all('div')[1].find('div').find_all('a')[2].get('href'))
            ln.append('https://research.commsec.com.au'+product.find_all('td')[3].find('a').get('href'))
            print(ln)
            mat.append(ln)
        index = index + 1
    return mat

In [11]:
def get_announce(excel_dir,sheet_name,browser,download_dir):
    df = pd.read_excel(excel_dir, sheet_name = sheet_name)
    key_works = ['placement','raising','SPP','share purchase plan', 'IPO', 'Right issue','raises']
    file_index = 1
    find_placement = 0
    total_placement = [['time', 'time_url']]
    file_name_pdf = 'Trading-Halt ({file_index}).PDF'

    for i in range (0,len(df)):
        get_work = 0
        last_pdf = ''
        now_pdf = ''
        url_ann = ''
        placement = ''
        palce = []
        print(i+1)
        print(df.iloc[i][6])
        url = df.iloc[i][6]
        browser.get(url)
        time.sleep(3)
        kw_input = browser.find_element(By.XPATH, '/html/body/app-root/div/div/sub-nav/nav/ul/li[2]/a')
        kw_input.click()
        time.sleep(1)
        kw_input = browser.find_element(By.XPATH, '//*[@id="announcements-container"]/section/div[1]/div[1]/header/section/div/div/section/div/label')
        kw_input.click()
        time.sleep(1)
        kw_input = browser.find_element(By.XPATH, '//*[@id="announcements-container"]/section/div[1]/div[1]/header/section/div/div/section/div/ul/li[4]')
        kw_input.click()
        time.sleep(2)
        soup = BeautifulSoup(browser.page_source, 'lxml')
        table_ann = soup.find('div',class_='overflowable col-12')
        announcements = table_ann.find('tbody').find_all('tr')
        for product in announcements:
            get_broker = 0
            description = product.find_all('td')[2].text.strip()
    #         print(description)
            for word in key_works:
                if word in description.lower():
                    find_placement = 6
                    placement = product.find_all('td')[2].find('a').get('href')
    #                 print(placement)
                    break
            find_placement = find_placement - 1
            if 'trading halt'in description.lower():
                print(product.find_all('td')[0].text)
                p_time = product.find_all('td')[0].text
                url_ann = product.find_all('td')[2].find('a').get('href')
    #             print(url_ann)
                try:
                    last_pdf = get_last_file(download_dir)
                except Exception as error:
                    print("An exception occurred:", error)
                browser.get(url_ann)
                path = os.listdir(download_dir)
                check_file(file_name_pdf ,file_index, path, download_dir, last_pdf)
                pdf_name = file_name_pdf.format(file_index = str(file_index))
                pdf_path = os.path.join(download_dir, pdf_name)
                print(pdf_path)
                file_index = file_index + 1
                get_work = find_word(pdf_path,key_works)
                if get_work == 1:
                    if find_placement > 0:
                        print(placement)
                        palce.append(p_time)
                        palce.append(placement)
                        total_placement.append(palce)
                        break

        if get_work == 0 or find_placement <=0:
            total_placement.append(palce)

        time.sleep(2)
    return total_placement

In [21]:
def get_cash_debt(path):
    inside1 = []
    with pdfplumber.open(path) as pdf:
        cash = None
        debt = None
        for page in pdf.pages:
            tables = page.extract_tables()
            try:
                for table in tables:
                    if 'Reconciliation' in table[0][0]: 
                        print(table[-1][-2])
                        cash = table[-1][-2]
                    if 'Financing facilities' in table[0][0]: 
                        if(table[4][2] == '-' or table[4][2] == '' or table[4][2] == ' ' or table[4][2] == '–' or table[4][2] == 'Nil' ):
                            debt = 0
                        elif(table[4][2] != None):
                            debt = table[4][2]
                        print(table[4][2])  
            except Exception as error:
                print("An exception occurred:", error)
        print('cash '+str(cash).replace(',', ''))
        print('debt '+str(debt).replace(',', ''))
        debt = str(debt).replace(',', '')
        try:
            cash = str(cash).replace(',', '')
            cash = float(cash)/1000
        except Exception as error:
                print("An exception occurred:", error)
                cash = None
        try:
            if '-' not in debt:
                debt = float(debt)/1000
        except Exception as error:
                print("An exception occurred:", error)
                debt = None
 
        debt = str(debt)
        cash = str(cash)
        print(cash)
        print(debt)
        inside1.append(cash)
        inside1.append(debt)
        cash = '$'+str(cash)+'M'
        inside1.append(cash)
        print(inside1)
    return inside1

In [13]:
def catch_cash(excel_dir,sheet_name, browser, download_dir):
    df = pd.read_excel(excel_dir,sheet_name = sheet_name)
    cash_flow_pdf = 'Cash_Flow ({file_index}).PDF'
    total_list = [['cash','debt','cash']]
    file_index = 1
    last_pdf = ''
    for i in range (0,len(df)):
        print(i+1)
        print(df.iloc[i][7])
        url = df.iloc[i][7]
        try:
            last_pdf = get_last_file(download_dir)
        except Exception as error:
            print("An exception occurred:", error)
        browser.get(url)
        path = os.listdir(download_dir)
        check_file(cash_flow_pdf ,file_index, path, download_dir, last_pdf)
        pdf_name = cash_flow_pdf.format(file_index = str(file_index))
        pdf_path = os.path.join(download_dir, pdf_name)
        print(pdf_path)
        file_index = file_index + 1
        cashlist = get_cash_debt(pdf_path)
        total_list.append(cashlist)
    return total_list

open the web

In [14]:
chrome_options = Options()


download_dir = 'C:\\Users\\N\\Desktop'
file_name = 'ha001'
chromedriver_path = "C:\Program Files (x86)\Google\Chrome\Application\chromedriver"
password = 'Nc139687@NC1'
username = '78531231'
excel_downlaod_dir = "C:\\Users\\N\\Desktop"
excel_name = 'test11.xlsx'
excel_dir = os.path.join(excel_downlaod_dir, excel_name)
sheet_name = '01'
target_url = "https://www2.commsec.com.au/research/tools"


download_dir = os.path.join(download_dir, file_name)

os.makedirs(download_dir, exist_ok=True)

chrome_options.add_experimental_option("prefs", {
    "download.default_directory": download_dir, 
    "download.prompt_for_download": False, 
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True  
})

service = Service(executable_path = chromedriver_path)
options = webdriver.ChromeOptions()
browser = webdriver.Chrome(service=service, options=chrome_options)
time.sleep(3)

login_commec(browser,username,password)

browser.get(target_url)
time.sleep(2)
browser.find_element(By.XPATH, '//*[@id="module_2"]/ul/li[4]').click()
time.sleep(5)

Loading the page, interrupt this program after the page finalize the loading

In [25]:
while(True):
    time.sleep(2)
    browser.execute_script("window.scrollBy(0,document.body.scrollHeight)")

KeyboardInterrupt: 

catch info, load into excel

In [26]:
company_info = get_company(browser)

1
$0.016
13.00M
https://www2.commsec.com.au/quotes/?stockCode=HTG
HTG    Harvest Technology Group Ltd   https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzI1NjgtN01SREM3SUJWVkNSNEtJRENVQ05KSEhFMFUiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==  
['Harvest Technology Group Ltd', 'HTG', '$0.016', '$13.00M', 0.016, 13.0, 'https://www2.commsec.com.au/quotes/?stockCode=HTG', 'https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzI1NjgtN01SREM3SUJWVkNSNEtJRENVQ05KSEhFMFUiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==']
2
$0.198
52.00M
https://www2.commsec.com.au/quotes/?stockCode=PEK
PEK    Peak Rare Earths Limited   https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzI1NjctNk0yQ0pJUDVFR01NUDBBSlFFRFFJSFZBUUsiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==  
['Peak Rare Earths Limited', 'PEK', '$0.198', '$52.00M', 0.198, 52.0, 'https://www2.commsec.com.au/quotes/?stockCode=PEK', 'https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzI1NjctNk0yQ0p

$0.028
20.20M
https://www2.commsec.com.au/quotes/?stockCode=SUH
SUH    Southern Hemisphere Mining Limited   https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzI0NzUtNUtUVEpDVUNLMk5OMjk4VlE3TzU1S1RUREQiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==  
['Southern Hemisphere Mining Limited', 'SUH', '$0.028', '$20.20M', 0.028, 20.2, 'https://www2.commsec.com.au/quotes/?stockCode=SUH', 'https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzI0NzUtNUtUVEpDVUNLMk5OMjk4VlE3TzU1S1RUREQiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==']
18
$0.021
31.10M
https://www2.commsec.com.au/quotes/?stockCode=NGY
NGY    NuEnergy Gas Limited   https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzI0NzMtNlFFM0NWNzJSRjZPR1FPMVU4U0czT1AzS0oiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==  
['NuEnergy Gas Limited', 'NGY', '$0.021', '$31.10M', 0.021, 31.1, 'https://www2.commsec.com.au/quotes/?stockCode=NGY', 'https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzI0NzMtNlFF

$0.044
23.70M
https://www2.commsec.com.au/quotes/?stockCode=AAM
AAM    AuMEGA Metals Ltd   https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzIzODMtME42S0hDVUxES05EME1LUlZOVUtOOEhOREQiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==  
['AuMEGA Metals Ltd', 'AAM', '$0.044', '$23.70M', 0.044, 23.7, 'https://www2.commsec.com.au/quotes/?stockCode=AAM', 'https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzIzODMtME42S0hDVUxES05EME1LUlZOVUtOOEhOREQiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==']
34
$0.440
192.40M
https://www2.commsec.com.au/quotes/?stockCode=TLG
TLG    Talga Group Ltd   https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzIzODAtMzNWNTdBSjQ3UjNHN0pNQ1NRQ01QMjZMTjkiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==  
['Talga Group Ltd', 'TLG', '$0.440', '$192.40M', 0.44, 192.4, 'https://www2.commsec.com.au/quotes/?stockCode=TLG', 'https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzIzODAtMzNWNTdBSjQ3UjNHN0pNQ1NRQ01QMjZMTjkiLCJmaWQiOm

$0.087
12.50M
https://www2.commsec.com.au/quotes/?stockCode=TNY
TNY    Tinybeans Group Ltd   https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzIzMTEtN0E0OVFVTTJBTjNLRlNKMUlTSExQNlBFVlEiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==  
['Tinybeans Group Ltd', 'TNY', '$0.087', '$12.50M', 0.087, 12.5, 'https://www2.commsec.com.au/quotes/?stockCode=TNY', 'https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzIzMTEtN0E0OVFVTTJBTjNLRlNKMUlTSExQNlBFVlEiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==']
50
$1.140
81.80M
https://www2.commsec.com.au/quotes/?stockCode=LTP
LTP    LTR Pharma Ltd   https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzIzMDUtNEdQMDkwMENPQjUyOU1QVktIN1U2RkUxOFUiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==  
['LTR Pharma Ltd', 'LTP', '$1.140', '$81.80M', 1.14, 81.8, 'https://www2.commsec.com.au/quotes/?stockCode=LTP', 'https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzIzMDUtNEdQMDkwMENPQjUyOU1QVktIN1U2RkUxOFUiLCJmaWQiOm5

$0.003
4.23M
https://www2.commsec.com.au/quotes/?stockCode=IBG
IBG    Ironbark Zinc Ltd   https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzIyMzUtNkdPSENNQTYyOFNPQTRDVkpHMUNDTVJGVTQiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==  
['Ironbark Zinc Ltd', 'IBG', '$0.003', '$4.23M', 0.003, 4.23, 'https://www2.commsec.com.au/quotes/?stockCode=IBG', 'https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzIyMzUtNkdPSENNQTYyOFNPQTRDVkpHMUNDTVJGVTQiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==']
66
$0.026
2.23M
https://www2.commsec.com.au/quotes/?stockCode=OMX
OMX    Orange Minerals NL   https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzIyMzQtN0RETUZDN1NRRDgxT1NEMlM1RFNFOThTNUkiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==  
['Orange Minerals NL', 'OMX', '$0.026', '$2.23M', 0.026, 2.23, 'https://www2.commsec.com.au/quotes/?stockCode=OMX', 'https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzIyMzQtN0RETUZDN1NRRDgxT1NEMlM1RFNFOThTNUkiLCJmaWQiOm

$0.004
3.91M
https://www2.commsec.com.au/quotes/?stockCode=ADG
ADG    Adelong Gold Limited   https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzIxOTgtNklFVEtQUjZFRE5WRExSU1ZWUDI2NTRHS0MiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==  
['Adelong Gold Limited', 'ADG', '$0.004', '$3.91M', 0.004, 3.91, 'https://www2.commsec.com.au/quotes/?stockCode=ADG', 'https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzIxOTgtNklFVEtQUjZFRE5WRExSU1ZWUDI2NTRHS0MiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==']
82
$0.019
4.98M
https://www2.commsec.com.au/quotes/?stockCode=NMR
NMR    Native Mineral Resources Holdings Limited   https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzIxOTctMUxEUkszQ0pSU05RMUIwTDlIM1JBQlEzVDQiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==  
['Native Mineral Resources Holdings Limited', 'NMR', '$0.019', '$4.98M', 0.019, 4.98, 'https://www2.commsec.com.au/quotes/?stockCode=NMR', 'https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4Mz

$0.024
14.50M
https://www2.commsec.com.au/quotes/?stockCode=EPX
EPX    EP&T Global Limited   https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzIxMTAtM0tIRjFHNFBFMTBERUE5RTZPTkNRTFIxTzYiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==  
['EP&T Global Limited', 'EPX', '$0.024', '$14.50M', 0.024, 14.5, 'https://www2.commsec.com.au/quotes/?stockCode=EPX', 'https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzIxMTAtM0tIRjFHNFBFMTBERUE5RTZPTkNRTFIxTzYiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==']
98
$0.039
32.60M
https://www2.commsec.com.au/quotes/?stockCode=CXM
CXM    Centrex Limited   https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzIxMDMtN0xLTkE5RVFOVEtBRkxVOTNKOElMR1ZWTUUiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==  
['Centrex Limited', 'CXM', '$0.039', '$32.60M', 0.039, 32.6, 'https://www2.commsec.com.au/quotes/?stockCode=CXM', 'https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzIxMDMtN0xLTkE5RVFOVEtBRkxVOTNKOElMR1ZWTUUiLCJmaWQi

$0.620
26.60M
https://www2.commsec.com.au/quotes/?stockCode=CHW
CHW    Chilwa Minerals Ltd   https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzIwNTEtNzgwVUI4OVNKSVZUUjhTRlU5OFRWNDAxSkgiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==  
['Chilwa Minerals Ltd', 'CHW', '$0.620', '$26.60M', 0.62, 26.6, 'https://www2.commsec.com.au/quotes/?stockCode=CHW', 'https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzIwNTEtNzgwVUI4OVNKSVZUUjhTRlU5OFRWNDAxSkgiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==']
114
$0.140
37.50M
https://www2.commsec.com.au/quotes/?stockCode=YRL
YRL    Yandal Resources Limited   https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzIwNDktN1ZEUlNCUVYzQzJRSEg4M0tMMU83VTNQTUYiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==  
['Yandal Resources Limited', 'YRL', '$0.140', '$37.50M', 0.14, 37.5, 'https://www2.commsec.com.au/quotes/?stockCode=YRL', 'https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzIwNDktN1ZEUlNCUVYzQzJRSEg4M0tMMU8

In [32]:
def check_info_company(company_info):
    ch = 0
    for i in company_info:
        if len(i)!=8:
            print(i)
            ch = 1
    if ch == 0:
        print('no data lost')

In [33]:
check_info_company(company_info)

['Company Name', 'Ticker', 'Last price', 'Market Cap', 'Last price_d', 'Market Cap_d']


In [37]:
write_excel(company_info,excel_dir, sheet_name)

xlsx write successfully！


In [16]:
announce_info = get_announce(excel_dir,sheet_name, browser, download_dir)

1
https://www2.commsec.com.au/quotes/?stockCode=HTG
29/01/2024
An exception occurred: list index out of range
C:\Users\N\Desktop\ha001\Trading-Halt (1).PDF
https://cdn-api.markitdigital.com/apiman-gateway/CommSec/commsec-node-api/1.0/event/document/1410-02767489-4T2A81G6QGC6NRU6SOIM0IL1OO/pdf?access_token=0007rtw6H6go7MvxOI3yz9OS9CMf


D:\anaconda3\lib\site-packages\pypdfium2\_helpers\textpage.py:80: UserWarning: get_text_range() call with default params will be implicitly redirected to get_text_bounded()
  warnings.warn("get_text_range() call with default params will be implicitly redirected to get_text_bounded()")


2
https://www2.commsec.com.au/quotes/?stockCode=PEK
22/07/2024
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt.PDF.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt (2).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt.PDF.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt (2).PDF'
C:\Users\N\Desktop\ha001\Trading-Halt (2).PDF
07/08/2023
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt.PDF.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt (3).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt.PDF.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt (3).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt.PDF.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt (3).PDF'
C:\Us

An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt.PDF.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt (19).PDF'
C:\Users\N\Desktop\ha001\Trading-Halt (19).PDF
10
https://www2.commsec.com.au/quotes/?stockCode=SQX
29/08/2023
C:\Users\N\Desktop\ha001\Trading-Halt (20).PDF
11
https://www2.commsec.com.au/quotes/?stockCode=PR1
24/07/2024
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt.PDF.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt (21).PDF'
C:\Users\N\Desktop\ha001\Trading-Halt (21).PDF
22/08/2022
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt.PDF.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt (22).PDF'
C:\Users\N\Desktop\ha001\Trading-Halt (22).PDF
12
https://www2.commsec.com.au/quotes/?stockCode=RPG
13
https://www2.commsec.com.au/quotes/?stockCode=TCG
29/04/2024
An exception occurred: [WinE

An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt.PDF.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt (42).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt.PDF.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt (42).PDF'
C:\Users\N\Desktop\ha001\Trading-Halt (42).PDF
https://cdn-api.markitdigital.com/apiman-gateway/CommSec/commsec-node-api/1.0/event/document/1410-02660859-1JGQLLAU7L7QM25DKTGO19QU98/pdf?access_token=0007rtw6H6go7MvxOI3yz9OS9CMf
32
https://www2.commsec.com.au/quotes/?stockCode=TMZ
13/02/2023
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt.PDF.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt (43).PDF'
C:\Users\N\Desktop\ha001\Trading-Halt (43).PDF
33
https://www2.commsec.com.au/quotes/?stockCode=AAM
24/01/2024
C:\Users\N\Desktop\ha001\Trading-Halt (44).PDF
23/10/2023
An exception

...

An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt.PDF.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt (84).PDF'
C:\Users\N\Desktop\ha001\Trading-Halt (84).PDF
01/03/2023
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt.PDF.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt (85).PDF'
C:\Users\N\Desktop\ha001\Trading-Halt (85).PDF
57
https://www2.commsec.com.au/quotes/?stockCode=INF
08/05/2023
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt.PDF.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt (86).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt.PDF.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt (86).PDF'
C:\Users\N\Desktop\ha001\Trading-Halt (86).PDF
29/03/2023
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users

13/11/2023
C:\Users\N\Desktop\ha001\Trading-Halt (102).PDF
30/11/2022
An exception occurred: [WinError 2] 系统找不到指定的文件。: 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt.PDF.crdownload'
C:\Users\N\Desktop\ha001\Trading-Halt (103).PDF
https://cdn-api.markitdigital.com/apiman-gateway/CommSec/commsec-node-api/1.0/event/document/1410-02606963-3BDS4IPVPB2O9ISVJQJOPET1G6/pdf?access_token=0007rtw6H6go7MvxOI3yz9OS9CMf
70
https://www2.commsec.com.au/quotes/?stockCode=JAV
30/05/2024
C:\Users\N\Desktop\ha001\Trading-Halt (104).PDF
https://cdn-api.markitdigital.com/apiman-gateway/CommSec/commsec-node-api/1.0/event/document/1410-02813397-0M598FFBOED2IQI35GC64ALD35/pdf?access_token=0007rtw6H6go7MvxOI3yz9OS9CMf
71
https://www2.commsec.com.au/quotes/?stockCode=FCL
15/08/2023
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt.PDF.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt (105).PDF'
C:\Users\N\Desktop\ha001\Trading-Halt (105).PDF
https://cdn-

An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt.PDF.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt (118).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt.PDF.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt (118).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt.PDF.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt (118).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt.PDF.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt (118).PDF'
C:\Users\N\Desktop\ha001\Trading-Halt (118).PDF
82
https://www2.commsec.com.au/quotes/?stockCode=NMR
15/07/2024
C:\Users\N\Desktop\ha001\Trading-Halt (119).PDF
https://cdn-api.markitdigital.com/apiman-gateway/CommSec/commsec-node-api/1.0/event/document/1410-02828598-4JOV86K8ORNL8J4TSS5C7Q2G4

97
https://www2.commsec.com.au/quotes/?stockCode=EPX
01/02/2024
C:\Users\N\Desktop\ha001\Trading-Halt (138).PDF
https://cdn-api.markitdigital.com/apiman-gateway/CommSec/commsec-node-api/1.0/event/document/1410-02768381-6G7QF4I0H94S5906UNGKT8DSB2/pdf?access_token=0007rtw6H6go7MvxOI3yz9OS9CMf
98
https://www2.commsec.com.au/quotes/?stockCode=CXM
31/01/2024
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt.PDF.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt (139).PDF'
C:\Users\N\Desktop\ha001\Trading-Halt (139).PDF
https://cdn-api.markitdigital.com/apiman-gateway/CommSec/commsec-node-api/1.0/event/document/1410-02768731-5IS0CL2LR1J4K5573345H0BI2M/pdf?access_token=0007rtw6H6go7MvxOI3yz9OS9CMf
99
https://www2.commsec.com.au/quotes/?stockCode=B4P
100
https://www2.commsec.com.au/quotes/?stockCode=RKT
30/11/2023
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt.PDF.crdownload' ->

C:\Users\N\Desktop\ha001\Trading-Halt (161).PDF
https://cdn-api.markitdigital.com/apiman-gateway/CommSec/commsec-node-api/1.0/event/document/1410-02815958-0FO8SBS4BTBJ1CKKU9B25A4V4K/pdf?access_token=0007rtw6H6go7MvxOI3yz9OS9CMf
117
https://www2.commsec.com.au/quotes/?stockCode=GTE
08/08/2024
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt.PDF.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt (162).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt.PDF.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Trading-Halt (162).PDF'
C:\Users\N\Desktop\ha001\Trading-Halt (162).PDF
https://cdn-api.markitdigital.com/apiman-gateway/CommSec/commsec-node-api/1.0/event/document/1410-02837150-56F2N1IMUET2HSGF183IMKVBBP/pdf?access_token=0007rtw6H6go7MvxOI3yz9OS9CMf
118
https://www2.commsec.com.au/quotes/?stockCode=GDM
119
https://www2.commsec.com.au/quotes/?stockCode=HGO
22/02/2024
An exc

In [20]:
add_to_excel(announce_info, excel_dir, sheet_name)

xlsx write successfully！


In [23]:
cash_info = catch_cash(excel_dir,sheet_name, browser, download_dir)

1
https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzI1NjgtN01SREM3SUJWVkNSNEtJRENVQ05KSEhFMFUiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (1).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (1).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (1).PDF'
C:\Users\N\Desktop\ha001\Cash_Flow (1).PDF
cash None
debt None
An exception occurred: could not convert string to float: 'None'
An exception occurred: could not convert string to float: 'None'
None
None
['None', 'None', '$NoneM']
2
https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzI1NjctNk0yQ0pJUDVFR01NUDBBSlFFRFFJSFZBUU

An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (8).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (8).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (8).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (8).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (8).PDF'
C:\Users\N\Desktop\ha001\Cash_Flow (8).PDF
An exception occurred: argument of type 'NoneType' is not iterable
An exception occurred: argument of type 'NoneType' is n

An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (13).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (13).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (13).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (13).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (13).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (1

An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (19).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (19).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (19).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (19).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (19).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (1

An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (26).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (26).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (26).PDF'
C:\Users\N\Desktop\ha001\Cash_Flow (26).PDF
53
5,000
cash 53
debt 5000
0.053
5.0
['0.053', '5.0', '$0.053M']
27
https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzI0MTAtNjRONzNGME41SDFETVFMUVI0R1NVRzlNS1IiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==
C:\Users\N\Desktop\ha001\Cash_Flow (27).PDF
367
14,385
cash 367
debt 14385
0.367
14.385
['0.367', '14.385', '$0.367M']
28
https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzI0MDgtMjA2QVVBSzdESTQwVE5ETEM1UUw2STd

...

C:\Users\N\Desktop\ha001\Cash_Flow (89).PDF
2,217
-
cash 2217
debt 0
2.217
0.0
['2.217', '0.0', '$2.217M']
90
https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzIxNjQtMUVFNFNLMVRSRUQ0NjZVREE4U0Q4UkZFTTIiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (90).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (90).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (90).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (90).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\User

An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (96).PDF'
C:\Users\N\Desktop\ha001\Cash_Flow (96).PDF
5,261
Nil
cash 5261
debt 0
5.261
0.0
['5.261', '0.0', '$5.261M']
97
https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzIxMTAtM0tIRjFHNFBFMTBERUE5RTZPTkNRTFIxTzYiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (97).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (97).PDF'
C:\Users\N\Desktop\ha001\Cash_Flow (97).PDF
None
275
cash None
debt 275
An exception occurred: could not convert string to float: 'None'
None
0.275
['None', '0.275', '$NoneM']
98
https://research.commsec.com.au/LoadPDF?docKey=B64ENCey

An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (102).PDF'
C:\Users\N\Desktop\ha001\Cash_Flow (102).PDF
1,272
-
cash 1272
debt 0
1.272
0.0
['1.272', '0.0', '$1.272M']
103
https://research.commsec.com.au/LoadPDF?docKey=B64ENCeyJkayI6IjE0MTAtMDI4MzIwNzQtMUhCT1BROTdUTVNOS0ZISjQyT0k1RlREOUkiLCJmaWQiOm51bGwsImR0IjpudWxsfQ==
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (103).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (103).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (103).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:

An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (108).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (108).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (108).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (108).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (108).PDF'
C:\Users\N\Desktop\ha001\Cash_Flow (108).PDF
192
260
cash 192
debt 260
0.192
0.26
['0.192', '0.26', '$0.192M']
109
https://research.commsec.com.au/LoadPDF?

An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (118).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (118).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (118).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (118).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Flow (118).PDF'
An exception occurred: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\N\\Desktop\\ha001\\LoadPDF.pdf.crdownload' -> 'C:\\Users\\N\\Desktop\\ha001\\Cash_Fl

In [25]:
add_to_excel(cash_info, excel_dir, sheet_name)

xlsx write successfully！


In [ ]:
os._exit(0)